This is based on Fabian Pedregosas [blog](https://fa.bianp.net/blog/2013/logistic-ordinal-regression/) and deprecated [github](https://github.com/fabianp/minirank/blob/master/minirank/logistic.py)

In [1]:
import numpy as np
from numpy.random import default_rng
from sklearn import metrics
from sklearn.preprocessing import StandardScaler

from scipy import optimize, linalg, sparse

In [2]:
rng = default_rng(1234)

# Simulation Data

Use a subsample of the simulation data for testing


In [3]:
from masterthesis.data import load_simdata, load_acinar
data = load_simdata("/home/julian/Uni/MasterThesis/data/simdata.h5ad")

In [4]:
idx = data.var['Setting'] == "TS"  # extract only the time series samples
sim_X = data.X[:, idx]
sim_y = data.obs["Ordinal_Time_Labels"]

In [5]:
# random subsample genes
y_idx = rng.choice(np.arange(sim_y.size), size=sim_y.size // 2, replace=False)
x_idx = rng.choice(np.arange(sim_X.shape[1]), size=sim_X.shape[1] // 4, replace=False)
sim_X = sim_X[y_idx, :]
sim_X = sim_X[:, x_idx]
sim_y = sim_y[y_idx]

In [6]:
scaler = StandardScaler()
sim_X = scaler.fit_transform(sim_X)

In [7]:
print(sim_X.shape)
print(sim_y.shape)

(286, 1031)
(286,)


# Acinar Data

In [8]:
acinar_ann = load_acinar()

In [9]:
# selected Genes after preprocessing in R
sel_genes = ["REG3A", "AMY2A", "MT2A", "OLFM4",
             "SYCN", "CELA2B", "FGL1", "AMY2B",
             "MT1G", "TM4SF1", "CELA2A", "PDK4", 
             "TACSTD2", "CD44", "PNLIPRP2", "ALB", 
             "ERP27", "LDHA", "REG3G", "CTRL", "CLPS",
             "FOS", "HSPA8", "SERPINA3", "CELA3B", "CRP"]

In [10]:
from sklearn.model_selection import train_test_split

ac_y = np.array([int(x) for x in acinar_ann.obs.donor_age])
ac_label_conv = dict(zip(np.unique(ac_y), range(len(ac_y))))
ac_y = np.array([ac_label_conv[l] for l in ac_y])
k = len(np.unique(ac_y))

ac_X_train, ac_X_test, ac_y_train, ac_y_test = train_test_split(acinar_ann[:,sel_genes].X, ac_y, 
                                                                test_size=0.1, 
                                                                stratify=ac_y,
                                                                random_state=1234)

In [11]:
scaler = StandardScaler()
ac_X_train = scaler.fit_transform(ac_X_train)
ac_X_test = scaler.fit_transform(ac_X_test)

# Model definition

Define the model based on its Loss function, the gradient and the hessian.

In [27]:
BIG = 1e10
SMALL = 1e-12

In [12]:
def phi(t):
    """
    logistic function, returns 1 / (1 + exp(-t))
    """
    idx = t > 0
    out = np.empty(t.size, dtype=np.float)
    out[idx] = 1. / (1 + np.exp(-t[idx]))
    exp_t = np.exp(t[~idx])
    out[~idx] = exp_t / (1. + exp_t)
    return out

def log_logistic(t):
    """
    (minus) logistic loss function, returns log(1 / (1 + exp(-t)))
    """
    idx = t > 0
    out = np.zeros_like(t)
    out[idx] = np.log(1 + np.exp(-t[idx]))
    out[~idx] = (-t[~idx] + np.log(1 + np.exp(t[~idx])))
    return out

In [67]:
#CHANGED FROM THE ORIGINAL! (See FABIANS Github)
def f_obj(x0, X, y):
        """
        Objective function
        """
        w, theta_0 = np.split(x0, [X.shape[1]])
        theta_1 = np.roll(theta_0, 1)
        t0 = theta_0[y]
        z = np.diff(theta_0)

        Xw = X.dot(w)
        a = t0 - Xw
        b = t0[k1:] - X[k1:].dot(w)
        c = (theta_1 - theta_0)[y][k1:]

        if np.any(c > 0):
            return BIG

        loss = -np.log(1 - np.exp(c)).sum()

        loss += b.sum() + log_logistic(b).sum() \
            + log_logistic(a).sum() \
            + alpha * w.dot(w) - np.log(z).sum() # penalty

        return loss

In [17]:
def f_grad(x0, X, y):
        """
        Gradient of the objective function
        """
        w, theta_0 = np.split(x0, [X.shape[1]])
        theta_1 = np.roll(theta_0, 1)
        t0 = theta_0[y]
        t1 = theta_1[y]
        z = np.diff(theta_0)

        Xw = X.dot(w)
        a = t0 - Xw
        b = t0[k1:] - X[k1:].dot(w)
        c = (theta_1 - theta_0)[y][k1:]

        # gradient for w
        phi_a = phi(a)
        phi_b = phi(b)
        grad_w = -X[k1:].T.dot(phi_b) + X.T.dot(1 - phi_a) + alpha * w

        # gradient for theta
        idx = c > 0
        tmp = np.empty_like(c)
        tmp[idx] = 1. / (np.exp(-c[idx]) - 1)
        tmp[~idx] = np.exp(c[~idx]) / (1 - np.exp(c[~idx])) # should not need
        grad_theta = (E1 - E0)[:, k1:].dot(tmp) \
            + E0[:, k1:].dot(phi_b) - E0.dot(1 - phi_a)

        grad_theta[:-1] += 1. / np.diff(theta_0)
        grad_theta[1:] -= 1. / np.diff(theta_0)
        out = np.concatenate((grad_w, grad_theta))

        return out

In [18]:
def f_hess(x0, s, X, y):
        x0 = np.asarray(x0)
        w, theta_0 = np.split(x0, [X.shape[1]])
        theta_1 = np.roll(theta_0, 1)
        t0 = theta_0[y]
        t1 = theta_1[y]
        z = np.diff(theta_0)

        Xw = X.dot(w)
        a = t0 - Xw
        b = t0[k1:] - X[k1:].dot(w)
        c = (theta_1 - theta_0)[y][k1:]

        D = np.diag(phi(a) * (1 - phi(a)))
        D_= np.diag(phi(b) * (1 - phi(b)))
        D1 = np.diag(np.exp(-c) / (np.exp(-c) - 1) ** 2)
        Ex = (E1 - E0)[:, k1:].toarray()
        Ex0 = E0.toarray()
        H_A = X[k1:].T.dot(D_).dot(X[k1:]) + X.T.dot(D).dot(X)
        H_C = - X[k1:].T.dot(D_).dot(E0[:, k1:].T.toarray()) \
            - X.T.dot(D).dot(E0.T.toarray())
        H_B = Ex.dot(D1).dot(Ex.T) + Ex0[:, k1:].dot(D_).dot(Ex0[:, k1:].T) \
            - Ex0.dot(D).dot(Ex0.T)

        p_w = H_A.shape[0]
        tmp0 = H_A.dot(s[:p_w]) + H_C.dot(s[p_w:])
        tmp1 = H_C.T.dot(s[:p_w]) + H_B.dot(s[p_w:])
        return np.concatenate((tmp0, tmp1))

# Scipy Optimize fit

In [75]:
# change according to dataset currently used
X = ac_X_train
y = ac_y_train
#X = sim_X
#y = sim_y

# regularization
alpha = 0

# weird but important model stuff
unq_y = np.unique(y)
k1 = np.sum(y == unq_y[0])
E0 = (y[:, np.newaxis] == unq_y).astype(np.int)
E1 = np.roll(E0, -1, axis=-1)
E1[:, -1] = 0.
E0, E1 = map(sparse.csr_matrix, (E0.T, E1.T))

# fiting options
options = {'maxiter' : 1000, 'disp': 0, 'maxfun':10000}

# initial weights
x0 = rng.random(X.shape[1] + np.unique(y).size) / X.shape[1]

# fit
out = optimize.minimize(f_obj, x0, args=(X, y), method="Nelder-Mead", jac=f_grad, hessp=f_hess)

/tmp/ipykernel_167575/2606101717.py:13: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  E0 = (y[:, np.newaxis] == unq_y).astype(np.int)


In [76]:
out

 final_simplex: (array([[0.0001633 , 0.03156496, 0.03187995, ..., 0.01427545, 0.00545713,
        0.02498236],
       [0.00016355, 0.03156496, 0.03187995, ..., 0.01427545, 0.00545713,
        0.02498236],
       [0.0001633 , 0.03161428, 0.03187995, ..., 0.01427545, 0.00545713,
        0.02498236],
       ...,
       [0.0001633 , 0.03156496, 0.03187995, ..., 0.01429775, 0.00545713,
        0.02498236],
       [0.0001633 , 0.03156496, 0.03187995, ..., 0.01427545, 0.00546566,
        0.02498236],
       [0.0001633 , 0.03156496, 0.03187995, ..., 0.01427545, 0.00545713,
        0.02502139]]), array([1.e+10, 1.e+10, 1.e+10, 1.e+10, 1.e+10, 1.e+10, 1.e+10, 1.e+10,
       1.e+10, 1.e+10, 1.e+10, 1.e+10, 1.e+10, 1.e+10, 1.e+10, 1.e+10,
       1.e+10, 1.e+10, 1.e+10, 1.e+10, 1.e+10, 1.e+10, 1.e+10, 1.e+10,
       1.e+10, 1.e+10, 1.e+10, 1.e+10, 1.e+10, 1.e+10, 1.e+10, 1.e+10,
       1.e+10, 1.e+10, 1.e+10]))
           fun: 10000000000.0
       message: 'Optimization terminated successfully.'
  

## Gradient DIY

In [61]:
def phi(X, beta, theta):
    return 1 / (1 + np.matmul(beta.T, X) - theta)

In [145]:
X = np.arange(10).reshape(2,5)
y = np.array([1,2,0,3,4])
beta = np.arange(2)
theta = np.array([-0.5, -0.1, 0.0123, 0.56, 1])
k = theta.size
params = np.concatenate([beta, theta])
phi(X, beta, theta)

array([0.15384615, 0.14084507, 0.12519248, 0.11848341, 0.11111111])

In [65]:
def l1_regularizer(beta, l):
    return l * np.sum(np.abs(beta))

In [204]:
BIG = 10000000000

# log likelihood with regularization as objective function
def objective(params, X, y, k, lamb=100):
    beta = params[:-k]
    theta = params[-k:]
    theta1 = np.array([theta[i] for i in y])
    theta2 = np.array([theta[i-1] if i > 0 else -BIG for i in y]) 
    # fit term
    likelihood = -1 * np.sum(np.log(phi(X, beta, theta1) - phi(X, beta, theta2)))
    # regularization term
    likelihood += l1_regularizer(beta, lamb)
    return likelihood

In [179]:
def objective_grad(params, X, y, k):
    beta = params[:-k]
    theta = params[-k:]
    theta1 = np.array([theta[i] for i in y])
    theta2 = np.array([theta[i-1] if i > 0 else -BIG for i in y])
    
    beta_grad = np.sum(X * (1 - phi(X, beta, theta1) - phi(X, beta, theta2)),
                       axis=1)
    
    # first half of the gradient
    temp1 = 1 - phi(X, beta, theta) - (1 / (1 - np.exp(theta2 - theta1)))
    temp2 = 1 - phi(X, beta, theta) - (1 / (1 - np.exp(-1 * (theta2 - theta1))))
    theta_grad = temp1 + np.concatenate([[0], temp2[:-1]]) 

    return np.concatenate([beta_grad, theta_grad])

In [160]:
objective_grad(params, X, y, k)

[ 7.923339   27.09248244]
[-2.18709094 -5.67552231  9.14803831 -0.61490804  0.33236145]


/tmp/ipykernel_191274/2385301824.py:13: RuntimeWarning: overflow encountered in exp
  temp2 = 1 - phi(X, beta, theta) - (1 / (1 - np.exp(-1 * (theta2 - theta1))))


array([ 7.923339  , 27.09248244, -2.18709094, -5.67552231,  9.14803831,
       -0.61490804,  0.33236145])

In [171]:
objective(params, X, y, theta.size, 0.1)

23.12130433664053

In [205]:
from scipy.optimize import minimize

minimize(objective, params, args=(X,y,k), jac=objective_grad, method="SLSQP")

/tmp/ipykernel_191274/211562064.py:12: RuntimeWarning: overflow encountered in exp
  temp2 = 1 - phi(X, beta, theta) - (1 / (1 - np.exp(-1 * (theta2 - theta1))))
/tmp/ipykernel_191274/722998437.py:10: RuntimeWarning: invalid value encountered in log
  likelihood = -1 * np.sum(np.log(phi(X, beta, theta1) - phi(X, beta, theta2)))


     fun: nan
     jac: array([278.99167735, 815.50787302,  -5.51799945,  34.75405763,
       -17.72646518,  79.73272709,  76.10792446])
 message: 'Iteration limit reached'
    nfev: 1070
     nit: 100
    njev: 100
  status: 9
 success: False
       x: array([ 1.25061769, -0.29190542,  1.49156664,  1.64499377,  1.60628663,
        2.43035065,  1.82246599])

## Gradient Model v3 

Based on [mord by fabian](https://github.com/fabianp/mord/blob/master/mord/threshold_based.py)

In [ ]:

def sigmoid(t):
    # sigmoid function, 1 / (1 + exp(-t))
    # stable computation
    idx = t > 0
    out = np.zeros_like(t)
    out[idx] = 1. / (1 + np.exp(-t[idx]))
    exp_t = np.exp(t[~idx])
    out[~idx] = exp_t / (1. + exp_t)
    return out


def log_loss(Z):
    # stable computation of the logistic loss
    idx = Z > 0
    out = np.zeros_like(Z)
    out[idx] = np.log(1 + np.exp(-Z[idx]))
    out[~idx] = (-Z[~idx] + np.log(1 + np.exp(Z[~idx])))
    return out


def obj_margin(x0, X, y, alpha, n_class, weights, L, sample_weight):
    """
    Objective function for the general margin-based formulation
    """

    w = x0[:X.shape[1]]
    c = x0[X.shape[1]:]
    theta = L.dot(c)
    loss_fd = weights[y]

    Xw = X.dot(w)
    Alpha = theta[:, None] - Xw  # (n_class - 1, n_samples)
    S = np.sign(np.arange(n_class - 1)[:, None] - y + 0.5)

    err = loss_fd.T * log_loss(S * Alpha)
    if sample_weight is not None:
        err *= sample_weight
    obj = np.sum(err)
    obj += alpha * 0.5 * (np.dot(w, w))
    return obj


def grad_margin(x0, X, y, alpha, n_class, weights, L, sample_weight):
    """
    Gradient for the general margin-based formulation
    """

    w = x0[:X.shape[1]]
    c = x0[X.shape[1]:]
    theta = L.dot(c)
    loss_fd = weights[y]

    Xw = X.dot(w)
    Alpha = theta[:, None] - Xw  # (n_class - 1, n_samples)
    S = np.sign(np.arange(n_class - 1)[:, None] - y + 0.5)
    # Alpha[idx] *= -1
    # W[idx.T] *= -1

    Sigma = S * loss_fd.T * sigmoid(-S * Alpha)
    if sample_weight is not None:
        Sigma *= sample_weight

    grad_w = X.T.dot(Sigma.sum(0)) + alpha * w

    grad_theta = -Sigma.sum(1)
    grad_c = L.T.dot(grad_theta)
    return np.concatenate((grad_w, grad_c), axis=0)


def threshold_fit(X, y, alpha, n_class, mode='AE',
                  max_iter=1000, verbose=False, tol=1e-12,
                  sample_weight=None):
    """
    Solve the general threshold-based ordinal regression model
    using the logistic loss as surrogate of the 0-1 loss
    Parameters
    ----------
    mode : string, one of {'AE', '0-1', 'SE'}
    """

    X, y = check_X_y(X, y, accept_sparse='csr')
    unique_y = np.sort(np.unique(y))
    if not np.all(unique_y == np.arange(unique_y.size)):
        raise ValueError(
            'Values in y must be %s, instead got %s'
            % (np.arange(unique_y.size), unique_y))

    n_samples, n_features = X.shape

    # convert from c to theta
    L = np.zeros((n_class - 1, n_class - 1))
    L[np.tril_indices(n_class-1)] = 1.

    if mode == 'AE':
        # loss forward difference
        loss_fd = np.ones((n_class, n_class - 1))
    elif mode == '0-1':
        loss_fd = np.diag(np.ones(n_class - 1)) + \
            np.diag(np.ones(n_class - 2), k=-1)
        loss_fd = np.vstack((loss_fd, np.zeros(n_class - 1)))
        loss_fd[-1, -1] = 1  # border case
    elif mode == 'SE':
        a = np.arange(n_class-1)
        b = np.arange(n_class)
        loss_fd = np.abs((a - b[:, None])**2 - (a - b[:, None]+1)**2)
    else:
        raise NotImplementedError

    x0 = np.zeros(n_features + n_class - 1)
    x0[X.shape[1]:] = np.arange(n_class - 1)
    options = {'maxiter' : max_iter, 'disp': verbose}
    if n_class > 2:
        bounds = [(None, None)] * (n_features + 1) + \
                 [(0, None)] * (n_class - 2)
    else:
        bounds = None

    sol = optimize.minimize(obj_margin, x0, method='L-BFGS-B',
        jac=grad_margin, bounds=bounds, options=options,
        args=(X, y, alpha, n_class, loss_fd, L, sample_weight),
        tol=tol)
    if verbose and not sol.success:
        print(sol.message)

    w, c = sol.x[:X.shape[1]], sol.x[X.shape[1]:]
    theta = L.dot(c)
    return w, theta